## 程序说明
时间: 2018-01-25  
说明: 该程序是由一个 包含两个卷积层，一个池化层和一个全连接层组成的神经网络  
数据集: MNIST

## 加载keras模块

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)

In [2]:
from keras.datasets import mnist
from keras.models import  Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

## 绘制模型加载plot_model

In [3]:
from keras.utils.vis_utils import plot_model

## 变量初始化

In [4]:
batch_size = 128
nb_classes = 10
epochs = 12

# 输入数据的维度
img_rows, img_cols = 28, 28
# 使用的卷积滤波器的数量
nb_filters = 32
# 用于max pooing的池化面积
pool_size = (2, 2)
# 卷积核的尺寸
kernel_size = (3, 3)

## 准备数据

需要把数据还原成28*28大小的数组
> Theano 和tensorflow的维度表示的含义是不一样的
'th' 模式，即Theano模式会把100张RGB三通道的16*32(高16宽32）彩色图表示为下面这种形式（100,3,16,32)，caffe采取的也是这种方式。第0个维度是样本数，第1个维度是通道数，代表颜色通道数。后面两个就是高和宽了。
而tensorflow，即'tf'模式的表达形式是(100,16,32,3)，即把通道数放在了最后。这两个表达方法本质上没有什么区别。

In [6]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0],img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
X_train =  X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /=255
X_test /=255

In [7]:
X_train.shape

(60000, 28, 28, 1)

## 转换类标号

In [8]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## 建立模型

## 使用Sequential()

In [10]:
model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size=kernel_size, padding='valid', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size=kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

## 打印模型

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

## 绘制模型结构图,并保存图片

In [13]:
plot_model(model, to_file='model-cnn.png')

## 训练与评估

## 编译模型

In [14]:
model.compile(loss='categorical_crossentropy',
             optimizer='adadelta',
             metrics=['accuracy'])

## 迭代训练

In [15]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 15s 249us/step - loss: 0.3006 - acc: 0.9067 - val_loss: 0.0665 - val_acc: 0.9791
Epoch 2/12
60000/60000 [==============================] - 11s 184us/step - loss: 0.1037 - acc: 0.9693 - val_loss: 0.0442 - val_acc: 0.9861
Epoch 3/12
60000/60000 [==============================] - 11s 182us/step - loss: 0.0754 - acc: 0.9776 - val_loss: 0.0396 - val_acc: 0.9874
Epoch 4/12
60000/60000 [==============================] - 11s 182us/step - loss: 0.0635 - acc: 0.9813 - val_loss: 0.0331 - val_acc: 0.9887
Epoch 5/12
60000/60000 [==============================] - 11s 182us/step - loss: 0.0572 - acc: 0.9826 - val_loss: 0.0310 - val_acc: 0.9888
Epoch 6/12
60000/60000 [==============================] - 11s 182us/step - loss: 0.0509 - acc: 0.9849 - val_loss: 0.0310 - val_acc: 0.9896
Epoch 7/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0484 - acc: 0.9852 - val_loss: 0.

## 模型评估

In [16]:
score = model.evaluate(X_test,Y_test, verbose=0)

In [17]:
score

[0.02881160225927397, 0.991]